In [30]:
import requests as re
from bs4 import BeautifulSoup as bs
import os
from pathlib import Path

In [46]:
currDir = Path(os.getcwd())
dataDir = currDir / "Data" / "MDMA.txt" 
dataFile = open(dataDir, "r")

In [51]:
mdma = dataFile.read()

In [58]:
%matplotlib inline

import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests as rq
%reload_ext tensorboard

In [49]:
# make var of training data

In [53]:
# for TensorBoard
import datetime, os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

def makeNewTensorDir(): 
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = keras.callbacks.TensorBoard(logdir)
    return tensorboard_callback

In [55]:
# here
text = mdma

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: Digesting the input text. We break it down into a set of unique characters then sort them and make a dictionary of them
#    with indices as values. Also create a dict with keys and values flipped.  
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: It creates a 'sentences' list which are just 40 char blocks of the text. 
#   seqlen determines the number of chars in each 'sentence' and step is the same
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: x and y are 3D matrices, they have dimensions 'seqlen' (40 in our case), the num of sentences and the num of chars 
#   in the prev created set of unique chars from 'pulpfiction'. 
#  a value in this matrix then is able to represent a char in the entire 'pulpfiction' data with it's postion
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    # A3a: Populates the matrices
    #  a point in 'x' will indicate the char in the text, in 'y' the char that comes directly after it (in, out)
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
#  "Whether to return the last output in the output sequence, or the full sequence."
#  Dense layer so we specify the output to give us one of the characters used by the text, squeeze values btw 0-1
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # no clue    
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

In [45]:
data = re.get("https://www.erowid.org/experiences/exp.php?ID=12581")

s = bs(data.text, 'html.parser')
trimmed = s.find_all(True, class_="report-text-surround")
# for a in trimmed:
#     print(a.string)
x = trimmed[0].text


In [ ]:
# tokenizing for BERT
# check if there's a sentence ending char before a new line char
# if not, discard that 'sentence' (as it's not a sentence)